# Food-20コンペ 
～転移学習/ファインチューニング編～

In [ ]:
# # Googleドライブのマウント（Colab使いのみ）

# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/dlc/week3.5

In [ ]:
# 図表が使えるようにする

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## データの準備

### データの読み込み

In [ ]:
# 読み込み処理

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# 💡訓練データのパス
train_path = '../input/train/'

# データセット化
full_dataset = ImageFolder(train_path)

# サイズ確認
print(f'full size: {len(full_dataset)}')

### 訓練データと検証データの割合を決めよう

In [ ]:
# 訓練データと検証データに分割する

import torch

# 💡訓練データの割合
train_ratio = 0.8

# 各データの数を算出
train_size = int(train_ratio * len(full_dataset))
valid_size = len(full_dataset) - train_size

# ランダムに分割
train_dataset, valid_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, valid_size]
)

# サイズ確認
print(f'train: {len(train_dataset)}, valid: {len(valid_dataset)}')

### 前処理を定義しよう

参考：

* [Pytorch公式 - torchvision.transforms](https://pytorch.org/vision/stable/transforms.html)
* [torchvision で使える Transform まとめ](https://pystyle.info/pytorch-list-of-transforms/)

In [ ]:
# 前処理の定義

from torchvision import transforms

# 訓練データ用前処理
train_transform = transforms.Compose([
    # 💡色々試してみよう
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2009, 0.1984, 0.2023))
])

# 検証データ&テストデータ用前処理
valid_transform = transforms.Compose([
    # 💡色々試してみよう
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2009, 0.1984, 0.2023))
])

# 前処理の適用
train_dataset.dataset.transform = train_transform
valid_dataset.dataset.transform = valid_transform

### ミニバッチサイズを決めよう

In [ ]:
# DataLoaderを作成

from torch.utils.data import DataLoader

# 💡ミニバッチサイズ
batch_size = 128

# 訓練用データローダー
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

# 検証用データローダー
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
# 番号とクラスの辞書
class_label = {
    0 : 'apple_pie',
    1 : 'cheesecake',
    2 : 'chocolate_cake', 
    3 : 'donuts', 
    4 : 'french_fries',
    5 : 'french_toast',
    6 : 'fried_rice',
    7 : 'hamburger',
    8 : 'ice_cream',
    9 : 'miso_soup',
    10: 'oysters',
    11: 'pizza',
    12: 'ramen',
    13: 'sashimi',
    14: 'spaghetti_bolognese',
    15: 'spaghetti_carbonara',
    16: 'steak',
    17: 'sushi',
    18: 'takoyaki',
    19: 'waffles'
}

# データローダから画像とラベルのテンソルを取り出す
def showBatch(dl):
    plt.figure(figsize=(12, 12))
    for images, labels in dl:
        for i in range(5):
            plt.subplot(1, 5, i+1)
            plt.imshow(np.transpose(images[i].numpy(), (1, 2, 0)))
            plt.title(class_label[int(labels[i].numpy())])
        break

In [ ]:
# データの確認
showBatch(train_loader)

## 学習済みモデルの準備

### モデルを決めよう

参考：

* [torchvision.models](https://pytorch.org/vision/stable/models.html)

In [ ]:
# ネットワーク定義

from torchvision import models

# クラス数
num_classes = 20

# 💡モデル(Resnet34)をロード
model = models.resnet34(pretrained=True)

print(model)

In [ ]:
import torch
import torch.nn as nn

# 最終層を変更
model.fc = nn.Linear(512, num_classes)

# 💡転移学習ならFalse/ファインチューニングならTrue
FT = True
if FT == True:
    # 全レイヤーの重みを固定
    for param in model.parameters():
        param.requires_grad = False

    # 最終層だけ重み固定を解除
    last_layer = list(model.children())[-1]
    for param in last_layer.parameters():
        param.requires_grad = True
else:
    pass

# デバイスに転送
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print(model)

### 損失関数と最適化手法を決めよう

参考：

* [torch.nn - Loss Functions（損失関数）](https://pytorch.org/docs/stable/nn.html#loss-functions)
* [torch.optim - Algorithms（最適化手法）](https://pytorch.org/docs/stable/optim.html#algorithms)

In [ ]:
# 損失関数と最適化手法を設定

import torch
import torch.optim as optim

# 💡損失関数: 交差エントロピー
criterion = nn.CrossEntropyLoss()

# 💡最適化手法: Adam
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 学習回数を決めよう

In [ ]:
# 💡エポック数
num_epochs = 5

In [ ]:
# lossとacc記録用リスト
train_loss_list, train_acc_list, valid_loss_list, valid_acc_list = [], [], [], []

# 最小loss保存用
min_valid_loss = 10000000

# 重みファイル
CKPT_FILE = 'food20_tl_best.ckpt'

### いざ学習

In [ ]:
# 学習

from tqdm.notebook import tqdm

print ('Training start ...')

# 事前に行ったエポック数
ad_epochs = len(train_loss_list)
 
for epoch in tqdm(range(num_epochs), desc='Epoch'):
    
    # エポック数の通知
    print(f'Epoch [{ad_epochs+epoch+1}/{ad_epochs+num_epochs}]')
    
    # エポックごとに初期化
    train_loss, train_acc, valid_loss, valid_acc = 0, 0, 0, 0
    
    # ====== 訓練モード ======
    model.train()

    # ミニバッチの数実行
    for images, labels in tqdm(train_loader, leave=False, desc='Train'):
        # deviceへ転送
        images, labels = images.to(device), labels.to(device)
        
        # 勾配をリセット
        optimizer.zero_grad()
        
        # 順伝搬の計算
        outputs = model(images)
        
        # lossとaccの計算
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        acc = (outputs.max(1)[1] == labels).sum()
        train_acc += acc.item()
        
        # 逆伝搬の計算
        loss.backward()
        # 重みの更新
        optimizer.step()

        # lossとaccの平均を計算
        avg_train_loss = train_loss / len(train_loader.dataset)
        avg_train_acc = train_acc / len(train_loader.dataset)

    
    # ====== 検証モード ======
    model.eval()
    
    # 必要のない計算を停止
    with torch.no_grad():
        # ミニバッチの数実行
        for images, labels in tqdm(valid_loader, leave=False, desc='Valid'):
            # deviceへ転送      
            images, labels = images.to(device), labels.to(device)
            
            # モデル出力
            outputs = model(images)
            
            # lossとaccの計算
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            acc = (outputs.max(1)[1] == labels).sum()
            valid_acc += acc.item()
    
    # lossとaccの平均を計算
    avg_valid_loss = valid_loss / len(valid_loader.dataset)
    avg_valid_acc = valid_acc / len(valid_loader.dataset)
    
    # ログの出力
    print (
        f'train_loss: {avg_train_loss:.4f}, train_acc: {avg_train_acc:.4f}, ' \
        f'val_loss: {avg_valid_loss:.4f}, val_acc: {avg_valid_acc:.4f}' 
    )

    # 一番lossが低いエポックの重みを保存
    if avg_valid_loss < min_valid_loss:
        print('Save best model ...')
        min_valid_loss = avg_valid_loss
        torch.save(model.state_dict(), CKPT_FILE)

    print('====================')
     
    # グラフ表示用リストに保存
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)
    valid_loss_list.append(avg_valid_loss)
    valid_acc_list.append(avg_valid_acc)

In [ ]:
# グラフの表示
plt.figure()
plt.plot(range(1, len(train_loss_list)+1), train_loss_list, color='blue', linestyle='-', label='train_loss')
plt.plot(range(1, len(valid_loss_list)+1), valid_loss_list, color='green', linestyle='--', label='valid_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training and validation loss')
plt.grid()
 
plt.figure()
plt.plot(range(1, len(train_acc_list)+1), train_acc_list, color='blue', linestyle='-', label='train_acc')
plt.plot(range(1, len(valid_acc_list)+1), valid_acc_list, color='green', linestyle='--', label='valid_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training and validation accuracy')
plt.grid()

## 提出ファイルの準備

### テストデータの読み込み

In [ ]:
# Datasetクラス定義

import glob
from PIL import Image
from torch.utils.data import Dataset, DataLoader

class MyTestDataset(Dataset):
    def __init__(self, test_image_dir, transform=None):
        self.image_paths = sorted(glob.glob(test_image_dir + '/*.jpg'))
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)
        
        return image

In [ ]:
from torch.utils.data import DataLoader

# 💡訓練データのパス
test_path = '../input/test/'

# Dataset化
test_dataset = MyTestDataset(test_image_dir=test_path, transform=valid_transform)

# DataLoader化
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=1,
    shuffle=False
)

In [ ]:
# 学習したモデルの読み込み
best_model = models.resnet34()
best_model.fc = nn.Linear(512, num_classes)
best_model.load_state_dict(torch.load(CKPT_FILE))
best_model.to(device)

# 結果保存用
preds = []

# 評価モード
best_model.eval()

with torch.no_grad():
    for images in test_loader:
        # deviceへ転送      
        images = images.to(device)
        # モデル出力
        outputs = best_model.forward(images)
        # 結果保存
        preds.append((torch.argmax(outputs, dim=1)).to('cpu').numpy())

In [ ]:
import pandas as pd

out_df = pd.DataFrame()
out_df['ImageName'] = [f'{str(i).zfill(4)}.jpg' for i in range(1, len(preds)+1)]
out_df['Label'] = np.concatenate(preds, axis=0)

out_df.to_csv('submission2.csv', index=None)